In [1]:
# Tiktoken is an open-source Python library that breaks text into tokens
#Langchian_community sued to make returiever
#Used chromaDB for database

!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 37.7 MB/s eta 0:00:00
 

In [ ]:
#create .env and add OPENAI_API_KEY, LANGCHAIN_API_KEY

import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""

In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.5 MB/s eta 0:00:00


In [6]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.0 MB/s eta 0:00:00


In [8]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import pdfplumber


In [9]:

# Step 1: Improved PDF Parsing
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    return text


In [10]:
pdf_text = extract_text_from_pdf( "/content/137253305_bgaawavakhroaw0xoj05dgcq.pdf")

# Step 2: Chunking for Better Retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", " ", ""])
documents = text_splitter.create_documents([pdf_text])

# Step 3: Optimized Vector Store (FAISS for faster search)
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = FAISS.from_documents(documents, embedding=embedding_model)

# Step 4: Better Retrieval with Filters
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Step 5: Optimized QA Chain with Structured Prompting
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="refine")


In [11]:
# query = "Extract the name of the patient"
# response = qa.run(query)

# print("Extracted Patient Name:", response)

# Step 6: Structured Lab Test Extraction
query_lab_tests = """Identify all lab test names and list them as bullet points. Do not include additional explanations or descriptions or recommended lab tests,
only test names which were completed. Here is an example of the expected output:
["CBC", "Blood Glucose", "Lipid profile"]
"""


In [12]:
lab_tests_response = qa.run(query_lab_tests)


In [13]:
lab_tests_response

'- Cholesterol Total\n- Triglycerides\n- HDL Cholesterol\n- LDL Cholesterol\n- VLDL Cholesterol\n- Non-HDL Cholesterol'

In [14]:
# # Convert response into a structured list
extracted_lab_tests = [test.strip() for test in lab_tests_response.split("\n-") if test.strip()]
print("Extracted Lab Tests:", lab_tests_response)

Extracted Lab Tests: - Cholesterol Total
- Triglycerides
- HDL Cholesterol
- LDL Cholesterol
- VLDL Cholesterol
- Non-HDL Cholesterol


In [15]:
import pandas as pd

# Load CSV containing lab tests and ICD codes
icd_df = pd.read_csv("/content/ICD_11_Codes.csv")  # Replace with actual path

In [16]:
# Convert to dictionary format
lab_test_mapping = {row["Title"]: row["Code"] for _, row in icd_df.iterrows()}

In [17]:
lab_test_mapping

{'Cholera': '1A00',
 'Intestinal infection due to other Vibrio': '1A01',
 'Intestinal infections due to Shigella': '1A02',
 'Intestinal infections due to Escherichia coli': '1A03',
 'Enteropathogenic Escherichia coli infection': '1A03.0',
 'Enterotoxigenic Escherichia coli infection': '1A03.1',
 'Enteroinvasive Escherichia coli infection': '1A03.2',
 'Enterohaemorrhagic Escherichia coli infection': '1A03.3',
 'Intestinal infections due to other specified Escherichia coli': '1A03.Y',
 'Intestinal infections due to Escherichia coli unspecified': '1A03.Z',
 'Intestinal infections due to Clostridioides difficile': '1A04',
 'Intestinal infections due to Yersinia enterocolitica': '1A05',
 'Gastroenteritis due to Campylobacter': '1A06',
 'Typhoid fever': '1A07',
 'Typhoid peritonitis': '1A07.0',
 'Other specified typhoid fever': '1A07.Y',
 'Typhoid fever unspecified': '1A07.Z',
 'Paratyphoid fever': '1A08',
 'Infections due to other Salmonella': '1A09',
 'Salmonella enteritis': '1A09.0',
 'In

In [18]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Define prompt template
prompt_template = """
You are an expert in medical diagnostics. Below is a list of lab tests with their corresponding ICD codes:

{lab_test_mapping}

Now, given the following extracted lab test names from a report:
{extracted_tests}

Please map each extracted test to the closest matching ICD code from the given mapping. If no exact match exists, find the most semantically appropriate match.

Provide the output in the format:
- Extracted Test: [Name]
- Mapped ICD Test: [Closest Lab Test Name]
- ICD Code: [Code]
"""

# Convert dictionary to string for the prompt
lab_test_mapping_str = "\n".join([f"{k}: {v}" for k, v in lab_test_mapping.items()])

In [19]:
lab_test_mapping_str

'Cholera: 1A00\nIntestinal infection due to other Vibrio: 1A01\nIntestinal infections due to Shigella: 1A02\nIntestinal infections due to Escherichia coli: 1A03\nEnteropathogenic Escherichia coli infection: 1A03.0\nEnterotoxigenic Escherichia coli infection: 1A03.1\nEnteroinvasive Escherichia coli infection: 1A03.2\nEnterohaemorrhagic Escherichia coli infection: 1A03.3\nIntestinal infections due to other specified Escherichia coli: 1A03.Y\nIntestinal infections due to Escherichia coli unspecified: 1A03.Z\nIntestinal infections due to Clostridioides difficile: 1A04\nIntestinal infections due to Yersinia enterocolitica: 1A05\nGastroenteritis due to Campylobacter: 1A06\nTyphoid fever: 1A07\nTyphoid peritonitis: 1A07.0\nOther specified typhoid fever: 1A07.Y\nTyphoid fever unspecified: 1A07.Z\nParatyphoid fever: 1A08\nInfections due to other Salmonella: 1A09\nSalmonella enteritis: 1A09.0\nInfections due to other Salmonella in other organs: 1A09.Y\nSalmonella infection unspecified: 1A09.Z\nO

In [20]:


# Format the prompt
prompt = prompt_template.format(
    lab_test_mapping=lab_test_mapping_str,
    extracted_tests="\n".join(extracted_lab_tests))


In [21]:
prompt

'\nYou are an expert in medical diagnostics. Below is a list of lab tests with their corresponding ICD codes:\n\nCholera: 1A00\nIntestinal infection due to other Vibrio: 1A01\nIntestinal infections due to Shigella: 1A02\nIntestinal infections due to Escherichia coli: 1A03\nEnteropathogenic Escherichia coli infection: 1A03.0\nEnterotoxigenic Escherichia coli infection: 1A03.1\nEnteroinvasive Escherichia coli infection: 1A03.2\nEnterohaemorrhagic Escherichia coli infection: 1A03.3\nIntestinal infections due to other specified Escherichia coli: 1A03.Y\nIntestinal infections due to Escherichia coli unspecified: 1A03.Z\nIntestinal infections due to Clostridioides difficile: 1A04\nIntestinal infections due to Yersinia enterocolitica: 1A05\nGastroenteritis due to Campylobacter: 1A06\nTyphoid fever: 1A07\nTyphoid peritonitis: 1A07.0\nOther specified typhoid fever: 1A07.Y\nTyphoid fever unspecified: 1A07.Z\nParatyphoid fever: 1A08\nInfections due to other Salmonella: 1A09\nSalmonella enteritis:

In [23]:
lab_tests_response

'- Cholesterol Total\n- Triglycerides\n- HDL Cholesterol\n- LDL Cholesterol\n- VLDL Cholesterol\n- Non-HDL Cholesterol'

In [24]:
# # Convert response into a structured list
extracted_lab_tests = [test.strip() for test in lab_tests_response.split("\n-") if test.strip()]
print("Extracted Lab Tests:", lab_tests_response)

Extracted Lab Tests: - Cholesterol Total
- Triglycerides
- HDL Cholesterol
- LDL Cholesterol
- VLDL Cholesterol
- Non-HDL Cholesterol


In [33]:
# # Run with Claude Sonnet on Bedrock
# response = chat_model(prompt)

# # Print response
# print(response)

In [26]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.5 MB/s eta 0:00:00


In [27]:
import pandas as pd
from rapidfuzz import process, fuzz

# Load the ICD codes CSV file
icd_df = pd.read_csv("/content/ICD_11_Codes.csv")  # Replace with actual path

In [28]:
# Assuming the CSV has columns: "Lab Test Name" and "ICD Code"
icd_test_names = icd_df["Title"].tolist()

In [29]:
# Function to find the best match using fuzzy matching
def find_best_icd_match(test_name, icd_test_names, icd_df):
    match, score, index = process.extractOne(test_name, icd_test_names, scorer=fuzz.WRatio)
    if score > 80:  # Set threshold for good match (adjust if needed)
        return match, icd_df.iloc[index]["Code"], score
    else:
        return None, None, score

# Map extracted lab tests to ICD codes
mapped_tests = []
for test in extracted_lab_tests:
    best_match, icd_code, match_score = find_best_icd_match(test, icd_test_names, icd_df)
    mapped_tests.append({
        "Extracted Test Name": test,
        "Matched ICD Test Name": best_match,
        "ICD Code": icd_code,
        "Match Score": match_score
    })

In [30]:
# Convert to DataFrame and display results
mapped_df = pd.DataFrame(mapped_tests)

In [31]:
mapped_df

,Extracted Test Name,Matched ICD Test Name,ICD Code,Match Score
0,- Cholesterol Total,Total external ophthalmoplegia,9C81.3,85.500000
1,Triglycerides,None,None,77.419355
2,HDL Cholesterol,Cholesterol atheroembolism,BD53.4,85.500000
3,LDL Cholesterol,Cholesterol atheroembolism,BD53.4,85.500000
4,VLDL Cholesterol,Cholesterol atheroembolism,BD53.4,85.500000
5,Non-HDL Cholesterol,Cholesterol atheroembolism to kidneys,BD53.40,85.500000
